#Introdução

O objetivo desta atividade é desenvolver uma rede neural para a prevenção de fraudes em cartões de crédito, ajustando seus hiperparâmetros para melhorar o desempenho do modelo. Para isso, utilizaremos um conjunto de dados de transações de cartões de crédito, disponível na biblioteca Keras. Durante a atividade, serão exploradas técnicas de ajuste de hiperparâmetros e avaliados os resultados obtidos com diferentes configurações de rede.


OBS: para a ponderada eu inseri o arquivo com os dados no collab, para rodar o codigo, insira o arquivo e informe o caminho na parte de carregamento dos dadso

##Preparando o ambiente

In [34]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score
from keras.metrics import Precision, Recall, AUC


##Carregando os dados

In [35]:
# Carregar os dados
data = pd.read_csv('/content/creditcard.csv')


##Preparando os dados

Nesta etapa, é importante destacar que os dados já foram previamente tratados. Assim, a única preparação necessária será remover valores nulos (NaN) da coluna Class, que representa se a transação é fraudulenta ou não. Também fizemos a regularização das classes, esses procedimentos garantirão que os dados estejam prontos para treinar a rede neural de forma eficiente.

In [36]:
#Dropnado os nan
data = data.dropna(subset=['Class'])

###Dividindo os dados

Nessa etapa estamos dividindo os dados para treino e teste, aqui separamos nossa coluna target e escolhemos usar 80% da base para treino e 20% para teste

In [37]:
X = data.drop('Class', axis=1)  # Features
y = data['Class']  # Target


# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


###Regularização dos dados


A regularização dos dados foi um passo crucial na análise, especialmente considerando o desbalanceamento nas classes do conjunto de dados. Diversas técnicas foram testadas para mitigar esse problema, incluindo a duplicação de classes menores e métodos de oversampling e undersampling. No entanto, nenhuma dessas abordagens trouxe resultados satisfatórios. A duplicação apenas replicou informações existentes, enquanto o undersampling reduziu a quantidade de dados disponíveis, e o oversampling resultou em redundâncias que limitaram o aprendizado do modelo.

Diante do insucesso das técnicas iniciais, optamos por utilizar o SMOTE (Synthetic Minority Over-sampling Technique), que se mostrou mais eficaz ao gerar novas amostras sintéticas a partir das instâncias existentes, aumentando a diversidade do conjunto de dados. Embora o SMOTE tenha sido o método que mais se aproximou de um resultado esperado, ainda assim enfrentamos dificuldades em generalizar para as classes minoritárias, evidenciando que o desbalanceamento continuava a impactar negativamente a performance do modelo.

In [38]:
from imblearn.over_sampling import SMOTE

# Aplicar SMOTE para gerar amostras sintéticas da classe minoritária
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = (X_train, y_train)


##Modelo
Nesta etapa, estamos criando a arquitetura do modelo inicial da rede neural. O modelo é composto por uma camada de entrada, duas camadas ocultas com 32 e 16 neurônios, respectivamente, e uma camada de saída com uma ativação sigmoide para prever a probabilidade de fraude. O modelo é compilado utilizando a função de perda binary_crossentropy, o otimizador adam e as métricas de avaliação incluem accuracy, Precision, Recall e AUC.

In [39]:
from keras.layers import Dropout

def create_custom_model(input_dim, neurons1=32, neurons2=16, dropout_rate=0.5, optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons1, input_dim=input_dim, activation='relu'))
    model.add(Dropout(dropout_rate))  # Adicionando camada de Dropout após a primeira camada
    model.add(Dense(neurons2, activation='relu'))
    model.add(Dropout(dropout_rate))  # Adicionando camada de Dropout após a primeira camada

    model.add(Dense(1, activation='sigmoid'))  # Saída com ativação sigmoide para prever probabilidade
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy', Precision(), Recall(), AUC(name='auc')])  # Incluindo as métricas no modelo

    return model


### Craindo modelo
Nesta etapa, o modelo é criado utilizando 64 neurônios na primeira camada oculta e 32 na segunda, com o otimizador adam. O modelo é treinado com 15 épocas e um tamanho de lote de 10, utilizando os dados de treino e validação. Após o treinamento, as previsões são feitas sobre os dados de teste, e as métricas de desempenho são calculadas: acurácia, recall, F1-score e AUC-ROC, permitindo a avaliação da capacidade do modelo em detectar fraudes.

In [32]:
# Criar o modelo com os parâmetros ajustados
model = create_custom_model(input_dim=X_train_resampled.shape[1], neurons1=64, neurons2=32, optimizer='adam')

# Treinar o modelo com os dados balanceados
model.fit(X_train_resampled, y_train_resampled, epochs=15, batch_size=10, validation_data=(X_test, y_test))

# Avaliar o modelo
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Calcular as métricas
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob)

print("Modelo Ajustado com OverSampling - Acurácia: {:.4f}, Recall: {:.4f}, F1-Score: {:.4f}, AUC-ROC: {:.4f}".format(accuracy, recall, f1, roc_auc))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/15
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 68s 3ms/step - accuracy: 0.9927 - auc: 0.5126 - loss: 53.8240 - precision_36: 0.0077 - recall_36: 0.0257 - val_accuracy: 0.9983 - val_auc: 0.5000 - val_loss: 0.0136 - val_precision_36: 0.0000e+00 - val_recall_36: 0.0000e+00
Epoch 2/15
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 61s 3ms/step - accuracy: 0.9980 - auc: 0.4965 - loss: 0.1082 - precision_36: 0.0000e+00 - recall_36: 0.0000e+00 - val_accuracy: 0.9983 - val_auc: 0.5000 - val_loss: 0.0202 - val_precision_36: 0.0000e+00 - val_recall_36: 0.0000e+00
Epoch 3/15
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 78s 3ms/step - accuracy: 0.9981 - auc: 0.4986 - loss: 0.1407 - precision_36: 0.0000e+00 - recall_36: 0.0000e+00 - val_accuracy: 0.9983 - val_auc: 0.5000 - val_loss: 0.0245 - val_precision_36: 0.0000e+00 - val_recall_36: 0.0000e+00
Epoch 4/15
22785/22785 ━━━━━━━━━━━━━━━━━━━━ 60s 3ms/step - accuracy: 0.9979 - auc: 0.4992 - loss: 0.0788 - precision_36: 4.7209e-04 - recall_36: 9.0549e-05 - val_accuracy: 0.9983 - v

 ##Hiperparametrização

 Aqui, inicialmente foi tentado o uso do KerasClassifier para realizar a busca de hiperparâmetros, porém, devido a inúmeros erros na biblioteca, essa abordagem foi descartada. Como solução, foi implementado um loop manual para testar diferentes combinações de hiperparâmetros, incluindo a quantidade de neurônios nas camadas ocultas e o otimizador. Para cada combinação, o modelo foi criado e treinado, seguido do cálculo das métricas de desempenho, como acurácia, recall, F1-score e AUC-ROC. A melhor combinação de hiperparâmetros foi selecionada com base na maior acurácia obtida.

In [33]:
from imblearn.over_sampling import RandomOverSampler

# Definição dos hiperparâmetros a serem testados
param_distributions = {
    'neurons1': [8, 16 ],
    'neurons2': [16, 32],
    'optimizer': ['adam', 'sgd']
}

# Variáveis para armazenar os melhores resultados
best_score = 0
best_params = None
best_metrics = {}

# Aplicar o RandomOverSampler para balancear as classes
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Loop para testar diferentes combinações de hiperparâmetros
for neurons1 in param_distributions['neurons1']:
      for neurons2 in param_distributions['neurons2']:
          for optimizer in param_distributions['optimizer']:
            print(f"Testando combinação: neurons1={neurons1}, neurons2={neurons2}, optimizer={optimizer}")

            # Criar e treinar o modelo com os dados balanceados
            model = create_custom_model(input_dim=X_train_resampled.shape[1], neurons1=neurons1, neurons2=neurons2, optimizer=optimizer)
            history = model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_data=(X_test, y_test), verbose=0)

            # Previsão no conjunto de teste
            y_pred_prob = model.predict(X_test)  # Probabilidades
            y_pred = (y_pred_prob > 0.5).astype(int)  # Converte para classes binárias (0 ou 1)

            # Calcular as métricas
            accuracy = accuracy_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            roc_auc = roc_auc_score(y_test, y_pred_prob)

            print(f"Validação - Acurácia: {accuracy:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}, AUC-ROC: {roc_auc:.4f}")

            # Comparar e armazenar o melhor modelo
            if accuracy > best_score:
                best_score = accuracy
                best_params = {'neurons1': neurons1, 'neurons2': neurons2, 'optimizer': optimizer}
                best_metrics = {'accuracy': accuracy, 'recall': recall, 'f1_score': f1, 'auc_roc': roc_auc}

# Imprimir a melhor combinação de hiperparâmetros e suas métricas
print(f"Melhor Acurácia: {best_score} usando {best_params}")
print(f"Melhores métricas: {best_metrics}")


Testando combinação: neurons1=8, neurons2=16, optimizer=adam


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Validação - Acurácia: 0.9983, Recall: 0.0000, F1-Score: 0.0000, AUC-ROC: 0.4912
Testando combinação: neurons1=8, neurons2=16, optimizer=sgd


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
Validação - Acurácia: 0.9983, Recall: 0.0000, F1-Score: 0.0000, AUC-ROC: 0.5000
Testando combinação: neurons1=8, neurons2=32, optimizer=adam


...

1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Validação - Acurácia: 0.0017, Recall: 1.0000, F1-Score: 0.0034, AUC-ROC: 0.5000
Testando combinação: neurons1=16, neurons2=32, optimizer=adam


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Validação - Acurácia: 0.0055, Recall: 0.9796, F1-Score: 0.0034, AUC-ROC: 0.4917
Testando combinação: neurons1=16, neurons2=32, optimizer=sgd


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1781/1781 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
Validação - Acurácia: 0.9983, Recall: 0.0000, F1-Score: 0.0000, AUC-ROC: 0.5000
Melhor Acurácia: 0.9982795547909132 usando {'neurons1': 8, 'neurons2': 16, 'optimizer': 'adam'}
Melhores métricas: {'accuracy': 0.9982795547909132, 'recall': 0.0, 'f1_score': 0.0, 'auc_roc': 0.49119381151447633}


##Conclusão

Ao comparar os resultados com e sem o uso de hiperparâmetros ajustados, é possível observar que, embora o desempenho não tenha sido o ideal, o objetivo principal foi alcançado. Com os hiperparâmetros aplicados, o modelo apresentou uma acurácia de 0.9983, mas com recall e F1-score em 0.0, o que indica que o ajuste não conseguiu melhorar a detecção das classes minoritárias. O AUC-ROC também não apresentou grande melhora, passando de 0.4912 para 0.4934, sugerindo que o modelo ainda enfrenta dificuldades em diferenciar adequadamente as classes.

Apesar de os resultados finais não terem atingido o nível esperado, o mais importante foi a aplicação e compreensão dos hiperparâmetros e técnicas de regularização, como o SMOTE. Esse processo foi fundamental para absorver o conhecimento necessário sobre ajustes de modelos e tratamento de dados desbalanceados, proporcionando um aprendizado significativo, mesmo sem alcançar métricas ideais.